# Analysis on the companies.jsonl dataset

In [127]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

## Data cleaning

In [130]:
data = []
with open("companies_updated.jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f if line.strip()]
df = pd.json_normalize(data)
df = df.drop(columns=["div_yield",'weekly_prices', "P1M", "P6M", "P1Y"])

# Takes first element for columns that have lists as values
df = df.applymap(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)
df = df.replace("n.a.", np.nan)
df = df.replace("n.s.", np.nan)

# Removing non italian companies
df = df[df["isin"].str.startswith("IT")]
# df = df.drop(columns="isin")

df1 = df[df.isna().any(axis=1)]
from IPython.display import display, HTML

display(HTML('<div style="overflow-x: auto;">' + df1.to_html() + "</div>"))

/var/folders/y8/lh75q07s5v3483_pb255smrw0000gn/T/ipykernel_21778/2119519482.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)
/var/folders/y8/lh75q07s5v3483_pb255smrw0000gn/T/ipykernel_21778/2119519482.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace("n.a.", np.nan)
/var/folders/y8/lh75q07s5v3483_pb255smrw0000gn/T/ipykernel_21778/2119519482.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df 

,name,isin,ticker,market_cap,p_e,sector,sub_sector,revenues,ebitda,profit,assets,sh_funds,net_fin,ebitda_sales,ros,roa,roe,debt_equity,debt_ebitda,employees,tang_assets,shf_liabilities,working_capital,retained_earnings,sub_sectore
0,Alkemy,IT0005314635,ALK,6.282000e+07,53.18,Tecnologia,Information technology,1.150370e+08,1.191800e+07,3.535000e+06,1.303190e+08,4.771600e+07,1.810400e+07,10.10,5.84,5.29,7.41,0.63,2.53,448.0,7.859000e+06,0.67,1.859000e+07,1.133200e+07,NaN
1,Aedes,IT0005349052,AEDES,5.280000e+06,7.42,Immobiliare,REIT - Retail,6.800000e+04,-1.558000e+06,-1.580000e+06,8.589000e+06,7.816000e+06,-4.392000e+06,NaN,NaN,-19.54,-20.21,0.02,-0.08,2.0,7.290000e+05,11.86,7.153000e+06,3.391000e+06,NaN
2,Amplifon,IT0004056880,AMP,6.070000e+09,39.19,Salute,Distribuzione servizi medici,2.260084e+09,5.268490e+08,1.550250e+08,3.693215e+09,1.101678e+09,9.809630e+08,23.22,11.42,7.02,14.07,1.07,2.23,14379.0,6.996690e+08,450.00,-4.631820e+08,8.096430e+08,NaN
3,Autostrade Merid,IT0000084043,AUTME,1.138000e+07,7.03,Beni Industriali,Ingegneria e Costruzione,0.000000e+00,2.867000e+06,1.451000e+06,5.955000e+07,4.937700e+07,-2.809600e+07,42.84,NaN,4.79,2.94,0.02,0.40,2.0,3.450000e+05,9.22,5.356400e+07,0.000000e+00,NaN
6,Brunello Cucinelli,IT0004764699,BC,8.880000e+09,78.11,Beni di Consumo Ciclici,Beni di lusso,1.139420e+09,3.337510e+08,1.238090e+08,1.379880e+09,4.536130e+08,5.649240e+08,29.23,16.41,13.58,27.29,1.48,2.01,2623.0,7.241590e+08,0.50,1.730440e+08,0.000000e+00,NaN
11,Digitouch,IT0005089476,DGT,2.479000e+07,18.35,Servizi alla Comunicazione,Agenzie Pubblicitarie,4.199700e+07,6.901000e+06,1.994000e+06,5.554300e+07,1.885600e+07,1.158500e+07,16.35,8.98,6.83,10.57,0.91,2.49,339.0,2.427000e+06,0.56,5.391000e+06,1.666000e+06,NaN
12,Ecomembrane,IT0005543332,ECMB,1.899000e+07,100.99,Beni di Consumo Ciclici,Imballaggio e contenitori,1.282950e+07,1.191012e+06,1.175370e+06,2.241715e+07,1.400008e+07,-5.281158e+06,9.19,2.05,1.19,8.40,0.11,1.30,41.0,1.491212e+06,1.81,8.847719e+06,0.000000e+00,NaN
14,Ena,IT0005468290,ENA,7.110000e+06,NaN,Immobiliare,Servizi Immobiliari,9.628260e+08,3.034360e+08,1.127100e+08,2.335431e+09,1.218733e+09,3.032080e+08,30.00,17.07,7.39,9.25,0.43,1.74,4309.0,8.228360e+08,1320.00,8.774320e+08,8.255500e+07,NaN
15,Ferretti,IT0005383291,YACHT,9.627000e+08,11.15,Beni di Consumo Ciclici,Veicoli ricreazionali,1.134485e+09,1.623170e+08,8.350400e+07,1.602248e+09,8.396800e+08,-2.812400e+08,14.07,8.49,6.11,9.94,0.04,0.20,NaN,3.823460e+08,1.14,2.158470e+08,0.000000e+00,NaN
16,Franchetti,IT0005508574,FCH,5.957000e+07,34.66,Beni Industriali,Ingegneria e Costruzione,4.742811e+06,1.839393e+06,1.200337e+06,8.972574e+06,4.606532e+06,-1.556900e+05,37.56,NaN,18.11,26.06,0.45,1.13,16.0,8.948600e+04,1.12,3.599896e+06,1.905800e+04,NaN


## Altman Z-Score
$$
    Z = 1.2 X_1 + 1.4  X_2 + 3.3  X_3 + 0.6  X_4 + 1.0  X_5
$$

In [129]:
# Step 1: Calculate Total Liabilities
df["total_debt"] = df["debt_equity"] * df["sh_funds"]
df["total_liabilities"] = df["total_debt"] + (
    df["assets"] - df["sh_funds"] - df["total_debt"]
)

# Step 2: Compute Altman Z-Score Components
df["X1"] = df["working_capital"] / df["assets"]
df["X2"] = df["sh_funds"] / df["assets"]
df["X3"] = df["ebitda"] / df["assets"]  # Approximating EBIT with EBITDA
df["X4"] = df["market_cap"] / df["total_liabilities"]
df["X5"] = df["revenues"] / df["assets"]

# Step 3: Calculate Altman Z-Score
df["Altman_Z_Score"] = (
    (1.2 * df["X1"])
    + (1.4 * df["X2"])
    + (3.3 * df["X3"])
    + (0.6 * df["X4"])
    + (1.0 * df["X5"])
)
df

,name,isin,ticker,market_cap,p_e,sector,sub_sector,revenues,ebitda,profit,...,retained_earnings,sub_sectore,total_debt,total_liabilities,X1,X2,X3,X4,X5,Altman_Z_Score
0,Alkemy,IT0005314635,ALK,6.282000e+07,53.18,Tecnologia,Information technology,1.150370e+08,11918000.0,3535000.0,...,11332000.0,NaN,3.006108e+07,8.260300e+07,0.142650,0.366148,0.091453,0.760505,0.882734,2.324617
1,Aedes,IT0005349052,AEDES,5.280000e+06,7.42,Immobiliare,REIT - Retail,6.800000e+04,-1558000.0,-1580000.0,...,3391000.0,NaN,1.563200e+05,7.730000e+05,0.832809,0.910001,-0.181395,6.830530,0.007917,5.781005
2,Amplifon,IT0004056880,AMP,6.070000e+09,39.19,Salute,Distribuzione servizi medici,2.260084e+09,526849000.0,155025000.0,...,809643000.0,NaN,1.178795e+09,2.591537e+09,-0.125414,0.298298,0.142653,2.342239,0.611956,2.755175
3,Autostrade Merid,IT0000084043,AUTME,1.138000e+07,7.03,Beni Industriali,Ingegneria e Costruzione,0.000000e+00,2867000.0,1451000.0,...,0.0,NaN,9.875400e+05,1.017300e+07,0.899479,0.829169,0.048144,1.118647,0.000000,3.070277
6,Brunello Cucinelli,IT0004764699,BC,8.880000e+09,78.11,Beni di Consumo Ciclici,Beni di lusso,1.139420e+09,333751000.0,123809000.0,...,0.0,NaN,6.713472e+08,9.262670e+08,0.125405,0.328734,0.241870,9.586869,0.825738,7.986742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817,Somec,IT0005329815,SOM,8.211000e+07,37.17,Beni Industriali,Ingegneria e Costruzione,3.676580e+08,10612000.0,-10374000.0,...,0.0,NaN,1.534345e+08,3.078840e+08,-0.052785,0.063741,0.032271,0.266691,1.118028,1.410432
818,Spindox,IT0005449522,SPN,5.467000e+07,100.52,Tecnologia,Information technology,9.681339e+07,6526720.0,254946.0,...,6605054.0,NaN,1.918820e+07,6.749586e+07,0.217965,0.185289,0.078781,0.809976,1.168589,2.435514
819,Star7,IT0005466195,STAR7,5.670000e+07,28.65,Beni Industriali,Servizi Aziendali Speciali,1.044056e+08,14550930.0,2231713.0,...,0.0,NaN,2.677365e+07,7.381640e+07,0.181022,0.301575,0.137676,0.768122,0.987850,2.542485
820,Sys-Dat,IT0005595423,SYS,1.861000e+08,36.74,Tecnologia,Information technology,2.371700e+07,5197000.0,2383000.0,...,0.0,NaN,1.319336e+07,2.512300e+07,0.170426,0.371092,0.130097,7.407555,0.593712,6.191605


## Computing features
$$
    \text{EBITDA Margin} = \frac{\text{EBITDA}}{\text{Revenues}}
$$
$$
    \text{Net Profit Margin} = \frac{\text{Profit}}{\text{Revenues}}
$$
$$
    \text{Net Profit Margin} = \frac{\text{Profit}}{\text{Revenues}}
$$